## Import libraries

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
token = os.getenv("token")

## Query: Companies that matches preferences - FOURSQUARE

- **Condition 3.** The CEO is vegan

- **Condition 4.** 30% of the company staff have at least 1 child

- **Condition 5.** Executives like Starbucks A LOT. Ensure there's a starbucks not too far

- **Condition 6.** Account managers need to travel a lot

- **Condition 7.** Everyone in the company is between 25 and 40, give them some place to go party

In [3]:
df = pd.read_csv('data/df_mongo.csv')

In [4]:
df.head()

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code
0,Digg,$45M,USA,CA,37.764726,-122.394523,135 Mississippi St,NaN,94107
1,GENWI,$7.1M,USA,CA,33.817100,-111.903500,4966 El Camino Real,Suite 200,94022
2,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001
3,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602
4,Sitemasher,C$5M,USA,FL,25.705057,-80.271378,NaN,NaN,NaN


In [5]:
def function_venue (venue, lat, lon, radius):
    
    
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&radius={radius}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response['results']

In [6]:
df['vegan_rest'] = df.apply(lambda row: function_venue("vegan", row['offices_latitude'], row['offices_longitude'], 500), axis=1)
df['num_vegan_rest'] = df['vegan_rest'].apply(lambda row: len(row))

In [7]:
df['preschool'] = df.apply(lambda row: function_venue("preschool", row['offices_latitude'], row['offices_longitude'], 500), axis=1)
df['num_preschool'] = df['preschool'].apply(lambda row: len(row))

In [8]:
df['starbucks'] = df.apply(lambda row: function_venue("starbucks", row['offices_latitude'], row['offices_longitude'], 500), axis=1)
df['num_starbucks'] = df['starbucks'].apply(lambda row: len(row))

In [9]:
df['clubs'] = df.apply(lambda row: function_venue('night%20clubs', row['offices_latitude'], row['offices_longitude'], 500), axis=1)
df['num_clubs'] = df['clubs'].apply(lambda row: len(row))

In [10]:
punctuation = {
    "vegan_rest": 8,
    "preschool": 5,
    "starbucks": 7,
    "club": 5
}

In [20]:
def weighs_function (punctuation):
    total = sum(punctuation.values())
    weights = {k: round(v/total*100, 2) for k, v in punctuation.items()}
    return weights

In [12]:
weights = weighs_function(punctuation)
print(weights)

{'vegan_rest': 32.0, 'preschool': 20.0, 'starbucks': 28.0, 'club': 20.0}


In [13]:
def function_weighted_punct (weights, col1, col2, col3, col4):
    weighted_punct = col1 * weights['vegan_rest'] \
                       + col2 * weights['preschool'] \
                       + col3 * weights['starbucks'] \
                       + col4 * weights['club']
    return weighted_punct

In [14]:
df['weighted_punct'] = df.apply(lambda row: function_weighted_punct(weights, row['num_vegan_rest'], row['num_preschool'], row['num_starbucks'], row['num_clubs']), axis=1)

In [15]:
df = df.sort_values(by='weighted_punct', ascending=False)

In [16]:
df

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,num_vegan_rest,preschool,num_preschool,starbucks,num_starbucks,clubs,num_clubs,weighted_punct
15,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,804.0
2,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001,"[{'fsq_id': '5f161e6f978e663e91ab7155', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",2,"[{'fsq_id': '55a555fd498e1727ae82a345', 'categ...",10,"[{'fsq_id': '4cc0b629f82ebfb7b4af6f91', 'categ...",8,800.0
3,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602,"[{'fsq_id': '4b02f734f964a520814b22e3', 'categ...",10,"[{'fsq_id': '52712a6911d25f0e0dfa94cb', 'categ...",1,"[{'fsq_id': '4aa01922f964a520483e20e3', 'categ...",10,"[{'fsq_id': '741d73901e18012f487f54d8', 'categ...",4,700.0
5,99designs,$35M,USA,CA,37.795531,-122.400598,447 Battery St.,3rd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '558a380a498e13330910aef6', 'categ...",1,"[{'fsq_id': '6167c8968b400716f1d65e0b', 'categ...",7,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7,676.0
8,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '5cc0a9cac0c0840025deceea', 'categ...",3,"[{'fsq_id': '49d0da91f964a520395b1fe3', 'categ...",3,"[{'fsq_id': '55ed1abb498e9e55d2e0edfc', 'categ...",9,644.0
9,Squarespace,$38.5M,USA,NY,40.720863,-74.003536,459 Broadway,5th Floor,10013,"[{'fsq_id': '5cd7104746e1b6002cc57a8c', 'categ...",10,"[{'fsq_id': '5723c0b4498e0709d3cee701', 'categ...",1,"[{'fsq_id': '4a6dea38f964a52052d31fe3', 'categ...",3,"[{'fsq_id': '527049ce11d2ef6b8356195f', 'categ...",8,584.0
14,Banyan Branch,$1.7M,USA,NY,40.720812,-74.003720,"455 Broadway, 3rd Floor",NaN,10013,"[{'fsq_id': '5cd7104746e1b6002cc57a8c', 'categ...",10,"[{'fsq_id': '5723c0b4498e0709d3cee701', 'categ...",1,"[{'fsq_id': '4a6dea38f964a52052d31fe3', 'categ...",3,"[{'fsq_id': '527049ce11d2ef6b8356195f', 'categ...",8,584.0
7,deviantART,$13.5M,USA,CA,34.101578,-118.333340,7095 Hollywood Blvd #788,NaN,90028,"[{'fsq_id': '4c57743e2308be9aa258596c', 'categ...",10,[],0,"[{'fsq_id': '4a3b3e02f964a5209ba01fe3', 'categ...",1,"[{'fsq_id': '4c01da4d716bc9b69fa0bc55', 'categ...",10,548.0
20,Talenthouse,$15.1M,USA,CA,34.080633,-118.388556,8810 Melrose Ave,NaN,90065,"[{'fsq_id': '52d613fd11d2dbc1f7b5b959', 'categ...",9,[],0,"[{'fsq_id': 'b85660e4fabc416868cdab9b', 'categ...",2,"[{'fsq_id': '4d33d70cb6093704dc8a05e0', 'categ...",6,464.0
13,Carrier IQ,$42M,USA,CA,37.395578,-122.083329,1200 Villa Street,NaN,94041,"[{'fsq_id': '53c2fe37498ef47b8dd63ab1', 'categ...",10,[],0,[],0,"[{'fsq_id': '4baf001df964a52035e63be3', 'categ...",3,380.0


In [17]:
df.to_csv('data/df_api.csv', index=False)